# Problem: Write a custom Dataset and Dataloader to load from a CSV file

### Problem Statement
You are tasked with creating a **custom Dataset** and **Dataloader** in PyTorch to load data from a given `data.csv` file. The loaded data will be used to run a pre-implemented linear regression model.

### Requirements
1. **Dataset Class**:
   - Implement a class `CustomDataset` that:
     - Reads data from a provided `data.csv` file.
     - Stores the features (X) and target values (Y) separately.
     - Implements PyTorch's `__len__` and `__getitem__` methods for indexing.

2. **Dataloader**:
   - Use PyTorch's `DataLoader` to create an iterable for batch loading the dataset.
   - Support user-defined batch sizes and shuffling of the data.

In [1]:
import torch
import pandas as pd

torch.manual_seed(42)
X = torch.rand(100, 1) * 10  # 100 data points between 0 and 10
y = 2 * X + 3 + torch.randn(100, 1)  # Linear relationship with noise

# Save the generated data to data.csv
data = torch.cat((X, y), dim=1)
df = pd.DataFrame(data.numpy(), columns=['X', 'y'])
df.to_csv('data.csv', index=False)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class LinearRegressionDataset(Dataset):
    def __init__(self, csv_file):
        # Load data from CSV file
        self.data = pd.read_csv(csv_file)
        self.X = torch.tensor(self.data['X'].values, dtype=torch.float32).view(-1, 1)
        self.y = torch.tensor(self.data['y'].values, dtype=torch.float32).view(-1, 1)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Example usage of the DataLoader
dataset = LinearRegressionDataset('data.csv')
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [7]:
# Define the Linear Regression Model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)  # Single input and single output

    def forward(self, x):
        return self.linear(x)

# Initialize the model, loss function, and optimizer
model = LinearRegressionModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    for batch_X, batch_y in dataloader:
        # Forward pass
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Log progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [100/1000], Loss: 1.1354
Epoch [200/1000], Loss: 0.4609
Epoch [300/1000], Loss: 1.4676
Epoch [400/1000], Loss: 0.3768
Epoch [500/1000], Loss: 1.1760
Epoch [600/1000], Loss: 0.9092
Epoch [700/1000], Loss: 0.3547
Epoch [800/1000], Loss: 0.6438
Epoch [900/1000], Loss: 0.3861
Epoch [1000/1000], Loss: 0.8075


In [8]:
# Display the learned parameters
[w, b] = model.linear.parameters()
print(f"Learned weight: {w.item():.4f}, Learned bias: {b.item():.4f}")

# Testing on new data
X_test = torch.tensor([[4.0], [7.0]])
with torch.no_grad():
    predictions = model(X_test)
    print(f"Predictions for {X_test.tolist()}: {predictions.tolist()}")

Learned weight: 1.9099, Learned bias: 3.1883
Predictions for [[4.0], [7.0]]: [[10.827767372131348], [16.55739974975586]]
